In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime 
%matplotlib inline
from sklearn import preprocessing
import warnings
warnings.filterwarnings(action = 'ignore')

### Dataset
This data approach student achievement in secondary education of two Portuguese schools. The data attributes include student grades, demographic, social and school-related features) and it was collected by using school reports and questionnaires. Two datasets are provided regarding the performance in two distinct subjects: Mathematics (mat) and Portuguese language (por).

I classified the students into three categories, "excellent", "average", and "poor", according to their final exam performance. Then I analyzed a few features that have significant influence on students' final performance, including using the internet as a learning resource,Romantic Status, Alcohol Consumption, Parents Education Level, etc. Finally, using available predictive features, I have tried various machine learning models to predict students' final performance classification and have compared models performance based on ROC index.

Dataset available at: http://archive.ics.uci.edu/ml/datasets/Student+Performance#

In [14]:
train_data = pd.read_csv('../data/student-mat.csv', sep=";")
train_data_1 = pd.read_csv('../data/student-por.csv', sep=";")

In [15]:
train_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [16]:
train_data_1.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [17]:
train_data.shape


(395, 33)

In [18]:
train_data_1.shape

(649, 33)

In [19]:
# merge datasets
student_df = pd.concat([train_data, train_data_1])

In [20]:
student_df.shape

(1044, 33)

In [21]:
student_df.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')

In [22]:
# rename column labels
student_df.columns = ['school','sex','age','address','family_size','parents_cohabitation_status','mother_education','father_education',
           'mother_job','father_job','reason','guardian','commute_time','study_time','failures','school_support',
          'family_support','paid_classes','activities','nursery','desire_higher_edu','internet','romantic','family_quality',
          'free_time','go_out_with_friends','weekday_alcohol_usage','weekend_alcohol_usage','health','absences','period_1_score','period_2_score','final_score']

In [23]:
#to look at the numerical fields and their describing mathematical values.
student_df.describe() 

,age,mother_education,father_education,commute_time,study_time,failures,family_quality,free_time,go_out_with_friends,weekday_alcohol_usage,weekend_alcohol_usage,health,absences,period_1_score,period_2_score,final_score
count,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000
mean,16.726054,2.603448,2.387931,1.522989,1.970307,0.264368,3.935824,3.201149,3.156130,1.494253,2.284483,3.543103,4.434866,11.213602,11.246169,11.341954
std,1.239975,1.124907,1.099938,0.731727,0.834353,0.656142,0.933401,1.031507,1.152575,0.911714,1.285105,1.424703,6.210017,2.983394,3.285071,3.864796
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,16.000000,2.000000,1.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,9.000000,9.000000,10.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,2.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,6.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


#### From the info available on the data, we can tell that the quality of the data is quite decent as there aren't any columns with null values and every cell has a single piece of data. This will significanlty simplify the processing stage of the data as we would not be required to compensate for null values or split dynamic data. However, there are many categorical fields in the data set and that requires some additional processing to generate better results from.

In [24]:
# look for missing values
student_df.isnull().sum()

school                         0
sex                            0
age                            0
address                        0
family_size                    0
parents_cohabitation_status    0
mother_education               0
father_education               0
mother_job                     0
father_job                     0
reason                         0
guardian                       0
commute_time                   0
study_time                     0
failures                       0
school_support                 0
family_support                 0
paid_classes                   0
activities                     0
nursery                        0
desire_higher_edu              0
internet                       0
romantic                       0
family_quality                 0
free_time                      0
go_out_with_friends            0
weekday_alcohol_usage          0
weekend_alcohol_usage          0
health                         0
absences                       0
period_1_s

In [ ]:
# look for the sum of missing values
student_df.isnull().sum().sum()

In [25]:
##checking for duplicates
student_df.duplicated().sum()

0

In [26]:
student_df.columns

Index(['school', 'sex', 'age', 'address', 'family_size',
       'parents_cohabitation_status', 'mother_education', 'father_education',
       'mother_job', 'father_job', 'reason', 'guardian', 'commute_time',
       'study_time', 'failures', 'school_support', 'family_support',
       'paid_classes', 'activities', 'nursery', 'desire_higher_edu',
       'internet', 'romantic', 'family_quality', 'free_time',
       'go_out_with_friends', 'weekday_alcohol_usage', 'weekend_alcohol_usage',
       'health', 'absences', 'period_1_score', 'period_2_score',
       'final_score'],
      dtype='object')

#### Transforming Values and Types
##### Some columns have numbers that represent categorical values. I'm going to change the name in some of these columns to make clearer what they mean.

##### I'm also going to change the type of the columns labeled numeric when they actually are categories

In [27]:
student_df['commute_time'] = student_df['commute_time'].map({1: '<15m', 2: '15-30m', 3: '30-1h', 4: '>1h'})

student_df['study_time'] = student_df['study_time'].map({1: '<2h', 2: '2-5h', 3: '5-10h', 4: '>10h'})

In [30]:
student_df[['mother_education','father_education','family_quality','go_out_with_friends','weekday_alcohol_usage','weekend_alcohol_usage','health']] = \
student_df[['mother_education','father_education','family_quality','go_out_with_friends','weekday_alcohol_usage','weekend_alcohol_usage','health']].astype('object')

#### Categorical Encoding
##### Linear Regression required that the attribute values be numerical. Therefore, columns with categorical data need to be encoded to a suitable numeric format. Attributes with 2 categories are encoded using binary encoding which converts the values to either 1 or 0. Attributes with more than 2 categories are encoded using one-hot encoding.

In [31]:
binary = ["sex", "family_size", "parents_cohabitation_status", "school_support", "family_support", "paid_classes", "activities", "internet", "romantic"]

In [33]:
multiple = ["mother_education", "father_education", "father_job", "mother_job", "reason"]

In [36]:
def binary_encoder(dataset, col):
    dataset[col] = dataset[col].astype('category')
    dataset[col] = dataset[col].cat.codes
    dataset[col] = dataset[col].astype('int')

In [34]:
df = pd.get_dummies(student_df, columns=multiple, prefix=multiple)

In [37]:
for col in binary:
    binary_encoder(df, col)

In [38]:
df.head()

,school,sex,age,address,family_size,parents_cohabitation_status,guardian,commute_time,study_time,failures,...,father_job_teacher,mother_job_at_home,mother_job_health,mother_job_other,mother_job_services,mother_job_teacher,reason_course,reason_home,reason_other,reason_reputation
0,GP,0,18,U,0,0,mother,15-30m,2-5h,0,...,1,1,0,0,0,0,1,0,0,0
1,GP,0,17,U,0,1,father,<15m,2-5h,0,...,0,1,0,0,0,0,1,0,0,0
2,GP,0,15,U,1,1,mother,<15m,2-5h,3,...,0,1,0,0,0,0,0,0,1,0
3,GP,0,15,U,0,1,mother,<15m,5-10h,0,...,0,0,1,0,0,0,0,1,0,0
4,GP,0,16,U,0,1,father,<15m,2-5h,0,...,0,0,0,1,0,0,0,1,0,0


In [39]:
df.dtypes

school                         object
sex                             int32
age                             int64
address                        object
family_size                     int32
parents_cohabitation_status     int32
guardian                       object
commute_time                   object
study_time                     object
failures                        int64
school_support                  int32
family_support                  int32
paid_classes                    int32
activities                      int32
nursery                        object
desire_higher_edu              object
internet                        int32
romantic                        int32
family_quality                 object
free_time                       int64
go_out_with_friends            object
weekday_alcohol_usage          object
weekend_alcohol_usage          object
health                         object
absences                        int64
period_1_score                  int64
period_2_sco

In [40]:
df.shape

(1044, 52)

In [41]:
# convert final_score to categorical variable 
# Excellent:15~20 Average:10~14 Poor:0~9
df['final_grade'] = 'na'
df.loc[(df.final_score >= 15) & (df.final_score <= 20), 'final_grade'] = 'excellent' 
df.loc[(df.final_score >= 10) & (df.final_score <= 14), 'final_grade'] = 'average' 
df.loc[(df.final_score >= 0) & (df.final_score <= 9), 'final_grade'] = 'poor' 
df.head(5)

,school,sex,age,address,family_size,parents_cohabitation_status,guardian,commute_time,study_time,failures,...,mother_job_at_home,mother_job_health,mother_job_other,mother_job_services,mother_job_teacher,reason_course,reason_home,reason_other,reason_reputation,final_grade
0,GP,0,18,U,0,0,mother,15-30m,2-5h,0,...,1,0,0,0,0,1,0,0,0,poor
1,GP,0,17,U,0,1,father,<15m,2-5h,0,...,1,0,0,0,0,1,0,0,0,poor
2,GP,0,15,U,1,1,mother,<15m,2-5h,3,...,1,0,0,0,0,0,0,1,0,average
3,GP,0,15,U,0,1,mother,<15m,5-10h,0,...,0,1,0,0,0,0,1,0,0,excellent
4,GP,0,16,U,0,1,father,<15m,2-5h,0,...,0,0,1,0,0,0,1,0,0,average


In [42]:
df.head()

,school,sex,age,address,family_size,parents_cohabitation_status,guardian,commute_time,study_time,failures,...,mother_job_at_home,mother_job_health,mother_job_other,mother_job_services,mother_job_teacher,reason_course,reason_home,reason_other,reason_reputation,final_grade
0,GP,0,18,U,0,0,mother,15-30m,2-5h,0,...,1,0,0,0,0,1,0,0,0,poor
1,GP,0,17,U,0,1,father,<15m,2-5h,0,...,1,0,0,0,0,1,0,0,0,poor
2,GP,0,15,U,1,1,mother,<15m,2-5h,3,...,1,0,0,0,0,0,0,1,0,average
3,GP,0,15,U,0,1,mother,<15m,5-10h,0,...,0,1,0,0,0,0,1,0,0,excellent
4,GP,0,16,U,0,1,father,<15m,2-5h,0,...,0,0,1,0,0,0,1,0,0,average


#### Next up, we will take the fields (columns) one by one to analyze their importance and effect on the final score value:

In [ ]:
#Plotting the distribution of the final grades.
sns.distplot(df['final_score']) 

#### From this we can tell that the distribution of the grades is decent and doesn't require any further skewness correction yet. We can go with this distribution for now to analyze the data and create a primitive model and it's error rate first. We can look into data processing of the G3 field afterwards if the results aren't satisfactory.

In [ ]:
corr = df.corr() # only works on numerical variables.
sns.heatmap(corr)

In [ ]:
print (corr['G3'].sort_values(ascending=False), '\n')

#### From the correlation graph above, we can look at the numerical fields to know the values that affect the end result the most. Obviously G2 and G1 are the most correlated fields to G3 as they are part of the calculation formula for G3 so they will have the greatest effect on our prediction. Another thing we can see is the negative correlation between failures and the G3 result. This also makes quite a lot of sense as more failures tend to negatively affect your end score. Absences and free time seem to not be very relevant in the dataset that are analyzing which can be a flag that may help us further understand the data in the future.

#### Now that we have analyzed the numerical data slightly and figured out the most correlated fields, we now have to take a look at the categorical data to figure out how useful the fields may be and how to introduce them into the prediction model. The simplest way to analyze those fields is to compare the means accross the categories.

In [ ]:
groupColumns = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup'
               , 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']

avgColumns = ['G3', 'G2', 'G1']

In [ ]:
school = data.groupby(groupColumns[0])[avgColumns].mean()
school.head()

#### From this, we see that Gabriel Pereira students generally do better than Mousinho da Silveira students. The same analysis can be done for a few more fields:

In [ ]:
sex = data.groupby(groupColumns[1])[avgColumns].mean()
sex.head()

#### Correlation between Attributes
##### The heatmap shows the correlation between different attributes. We can use it to find which attributes are highle correlated with the target label and select them whereas we can also drop the features that are highly correlated to other features

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(15,12))

ax = sns.heatmap(data=df.corr(), ax=ax, cmap="Blues")
ax.set_xlabel('Features',fontdict={"fontsize":16})
ax.set_ylabel('Features',fontdict={"fontsize":16})
ax.set_title('Correlation between different Features', loc="center", fontdict={"fontsize": 16, "fontweight":"bold"})

plt.savefig("heatmap.png", bbox_inches="tight")
plt.show()

##### From the above heatmap, columns G1, G2 and G3 are highly correlated to each other. The below plots show this correlation.

pairplot = sns.pairplot(dataset[["G1", "G2", "G3"]], palette="viridis")

plt.savefig("pairplot.png", bbox_inches="tight")
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,8))

ax[0] = sns.lineplot(x="G1", y="G3", data=dataset, palette="viridis", ax=ax[0])
ax[0].set_xlabel('G1',fontdict={"fontsize":16})
ax[0].set_ylabel('G3',fontdict={"fontsize":16})
ax[0].set_title('G3 vs G1', loc="center", fontdict={"fontsize": 16, "fontweight":"bold"})

ax[1] = sns.lineplot(x="G2", y="G3", data=dataset, palette="viridis", ax=ax[1])
ax[1].set_xlabel('G2',fontdict={"fontsize":16})
ax[1].set_ylabel('G3',fontdict={"fontsize":16})
ax[1].set_title('G3 vs G2', loc="center", fontdict={"fontsize": 16, "fontweight":"bold"})

plt.savefig("lineplot.png", bbox_inches="tight")
plt.show()